#2. Predição de preços de diamantes


Criação de modelos de regressão para predizer o valor do preço de um diamante usando os demais atributos como variáveis independentes.

Conjuntos de dados: diamonds.csv

Número de variáveis independentes: 11

Variável dependente: "price"

Modelos utilizados: LinearRegression, Lasso, DecisionTree, RandomForest, KNeighbors, GradientBoosting

##2.1 Importando bibliotecas

In [7]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

##2.2 Importação de dados

Importação dos arquivos de treino e teste disponibilizados no github através da biblioteca pandas

In [8]:
data_E2 = pd.read_csv("https://raw.githubusercontent.com/RodrigoSFMachado/Data_Science/main/data/diamonds.csv")

##2.3 Pre-processamento de dados

Observando a estrutura dos dados

In [9]:
data_E2.head(3)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31


Checando informações sobre os dados e verificando a tabela possui campos nulos

In [10]:
data_E2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


Separando a variável dependente (y) das variáveis independentes (X) nos dados de treino e teste

In [11]:
target_E2 = ['price']
y_E2 = data_E2[target_E2]
X_E2 = data_E2.drop(target_E2, axis=1)

As variáveis "cut", "color" e "clarity" são classificadas como variáveis categóricas ordinais. Portanto, elas foram categorizadas de acordo com as sequências adequadas de ordem de grandeza.

In [12]:
enconder_labels_E2 = {
    'cut' : ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal'],
    'color' : ['J', 'I', 'H', 'G', 'F', 'E', 'D'],
    'clarity' : ['I1', 'SI2', 'SI1', 'VS2', 'VS1','VVS2', 'VVS1', 'IF']

}

In [13]:
X_E2[enconder_labels_E2.keys()].head(3)

,cut,color,clarity
0,Ideal,E,SI2
1,Premium,E,SI1
2,Good,E,VS1


In [14]:


oe_E2 = OrdinalEncoder(categories = list(enconder_labels_E2.values()))

X_encoding_E2 = X_E2[list(enconder_labels_E2)]
X_encoding_E2 = oe_E2.fit_transform(X_encoding_E2)
X_encoding_E2 = pd.DataFrame(X_encoding_E2, columns= list(enconder_labels_E2))


In [15]:
X_encoded_E2 = X_E2.drop(X_E2[list(enconder_labels_E2)], axis = 1)
X_encoded_E2 = X_encoded_E2.join(X_encoding_E2)
X_encoded_E2.head(3)


,carat,depth,table,x,y,z,cut,color,clarity
0,0.23,61.5,55.0,3.95,3.98,2.43,4.0,5.0,1.0
1,0.21,59.8,61.0,3.89,3.84,2.31,3.0,5.0,2.0
2,0.23,56.9,65.0,4.05,4.07,2.31,1.0,5.0,4.0


Método two-way holdout: Separação dos dados em conjuntos de teste (20%) e treinamento (80%) utilizando a função "train_test_split".

In [16]:
X_train_E2, X_test_E2, y_train_E2, y_test_E2 = train_test_split(X_encoded_E2, y_E2, test_size = 0.2, random_state = 0)


In [17]:
X_train_E2 = X_train_E2.reset_index(drop= True)
X_test_E2 = X_test_E2.reset_index(drop = True)

Realizando o escalonamento dos dados numéricos para mitigar discrepâncias entre as faixas de valores.

In [18]:
standard_labels_E2 = ['carat', 'depth', 'table', 'x', 'y', 'z']

In [19]:
X_train_E2[standard_labels_E2].head(3)

,carat,depth,table,x,y,z
0,1.63,61.7,55.0,7.56,7.60,4.68
1,0.34,62.2,57.0,4.47,4.44,2.77
2,0.40,61.7,56.0,4.73,4.77,2.93


In [20]:
scaler_E2 = StandardScaler()

def data_train_standard_E2(dados):

  X_standard = dados[standard_labels_E2]
  X_standard = scaler_E2.fit_transform(X_standard)
  X_standard = pd.DataFrame(X_standard, columns= standard_labels_E2)

  X_droped = dados.drop(standard_labels_E2, axis=1)
  X_standard = X_droped.join(X_standard)

  return(X_standard)

def data_test_standard_E2(dados):

  X_standard = dados[standard_labels_E2]
  X_standard = scaler_E2.transform(X_standard)
  X_standard = pd.DataFrame(X_standard, columns= standard_labels_E2)

  X_droped = dados.drop(standard_labels_E2, axis=1)
  X_standard = X_droped.join(X_standard)

  return(X_standard)

In [21]:
X_train_final_E2 = data_train_standard_E2(X_train_E2)
X_test_final_E2 = data_test_standard_E2(X_test_E2)

In [22]:
X_train_final_E2[standard_labels_E2].head(3)

,carat,depth,table,x,y,z
0,1.752033,-0.036583,-1.100099,1.628446,1.621835,1.609092
1,-0.965068,0.312362,-0.203888,-1.123365,-1.126106,-1.084595
2,-0.838691,-0.036583,-0.651994,-0.891821,-0.839137,-0.858946


##2.4 Criando/treinando/avaliando modelo

In [23]:
models_E2 = {
    'lr' : (LinearRegression()),
    'lasso' : (Lasso()),
    'dtr' : (DecisionTreeRegressor()),
    'rfr' : (RandomForestRegressor()),
    'knnr' : (KNeighborsRegressor()),
    'gbr' : (GradientBoostingRegressor())
}

Treinamento dos modelos de regressão listados e avaliação através das métricas: MSE(mean-squared-error)/r²_score

In [24]:
y_train_temp_E2 = np.ravel(y_train_E2)
y_test_temp_E2 = np.ravel(y_test_E2)

for name , model in models_E2.items():

    model.fit(X_train_final_E2,y_train_temp_E2)

    MSE = mean_squared_error(y_test_temp_E2 , model.predict(X_test_final_E2))
    r2 = r2_score(y_test_temp_E2, model.predict(X_test_final_E2))

    print(f'\nModel: {model} \n    MSE_score: {MSE:.0f} \n    R²_score: {r2:.2f}')




Model: LinearRegression() 
    MSE_score: 1435761 
    R²_score: 0.91

Model: Lasso() 
    MSE_score: 1436719 
    R²_score: 0.91

Model: DecisionTreeRegressor() 
    MSE_score: 497658 
    R²_score: 0.97

Model: RandomForestRegressor() 
    MSE_score: 287149 
    R²_score: 0.98

Model: KNeighborsRegressor() 
    MSE_score: 508675 
    R²_score: 0.97

Model: GradientBoostingRegressor() 
    MSE_score: 381959 
    R²_score: 0.98
